In [ ]:
!pip install evaluate datasets optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 20.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import evaluate
import lightgbm as lgb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import softmax
from torch.optim import AdamW

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

from tqdm.notebook import tqdm, trange

from datasets import Dataset, Value

device = torch.device("cuda")

# 1. Подготовка данных

- Обрезаем тексты (ранее моделирование показало, что наилучшей дескриптивной способностью обладают тексты, начинающиеся с 1500 символа).
- Удаляем пустые тексты

In [ ]:
df10q = pd.read_csv('MDA_dataset_10K.csv', index_col=0)
df10q.dropna(inplace=True)

texts = df10q['MDA'].tolist()
labels = df10q['target_10_index'].tolist()

start_text = 1500

processed_texts = []
processed_labels = []
for text, label in zip(texts, labels):
    if len(text) > 0:
        if len(text) >= start_text:
            processed_text = text[start_text:]
        else:
            processed_text = text
        processed_texts.append(processed_text)
        processed_labels.append(label)

texts = processed_texts
labesls = processed_labels

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

# 2. Токенизация текстов

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")

tokenized_test = tokenized_test.remove_columns(["text"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")

tokenized_train = tokenized_train.cast_column("labels", Value("int64"))
tokenized_test = tokenized_test.cast_column("labels", Value("int64"))

train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_test, batch_size=8)

Map:   0%|          | 0/907 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/907 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

# 3. Подготовка модели к обучению

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=2, ignore_mismatched_sizes=True)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-6, fused=True)

num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# 4. Обучение модели

In [ ]:
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("roc_auc")

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_probs = []
    all_labels = []

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        probs = softmax(logits, dim=1)[:, 1].detach().cpu()
        labels = batch["labels"].detach().cpu()

        all_probs.extend(probs.numpy())
        all_labels.extend(labels.numpy())

    score = metric.compute(prediction_scores=all_probs, references=all_labels)
    print(f"Epoch {epoch}: ROC-AUC = {score['roc_auc']:.4f}")

  0%|          | 0/2280 [00:00<?, ?it/s]

Epoch 0: ROC-AUC = 0.4866
Epoch 1: ROC-AUC = 0.4963
Epoch 2: ROC-AUC = 0.4990
Epoch 3: ROC-AUC = 0.5054
Epoch 4: ROC-AUC = 0.5096
Epoch 5: ROC-AUC = 0.5087
Epoch 6: ROC-AUC = 0.5158
Epoch 7: ROC-AUC = 0.5220
Epoch 8: ROC-AUC = 0.5263
Epoch 9: ROC-AUC = 0.5318
Epoch 10: ROC-AUC = 0.5405
Epoch 11: ROC-AUC = 0.5449
Epoch 12: ROC-AUC = 0.5418
Epoch 13: ROC-AUC = 0.5439
Epoch 14: ROC-AUC = 0.5482
Epoch 15: ROC-AUC = 0.5507
Epoch 16: ROC-AUC = 0.5529
Epoch 17: ROC-AUC = 0.5512
Epoch 18: ROC-AUC = 0.5517
Epoch 19: ROC-AUC = 0.5508


In [ ]:
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

# 5. Обучение классификатора на эмбеддингах

In [ ]:
df10q = pd.read_csv('MDA_dataset_10K.csv', index_col=0)
df10q.dropna(inplace=True)

texts = df10q['MDA'].tolist()
labels = df10q['target_10_index'].tolist()

start_text = 1500

processed_texts = []
processed_labels = []
for text, label in zip(texts, labels):
    if len(text) > 0:
        if len(text) >= start_text:
            processed_text = text[start_text:]
        else:
            processed_text = text
        processed_texts.append(processed_text)
        processed_labels.append(label)

texts = processed_texts
labesls = processed_labels

def get_text_embedding(text):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs, output_hidden_states=True)
    hidden_states = outputs.hidden_states
    last_hidden = hidden_states[-1]
    cls_embedding = last_hidden[:, 0, :]
    return cls_embedding.detach().cpu().numpy()[0]

vectors = [get_text_embedding(text) for text in texts]

X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
def objective(trial):
    n_components = trial.suggest_float('n_components', 0.8, 0.9)

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(np.array(X_train))
    X_test_pca = pca.transform(np.array(X_test))

    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 5, 150),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12)
    }

    lgb_train = lgb.Dataset(X_train_pca, label=y_train)

    gbm = lgb.train(param, lgb_train, num_boost_round=1000)

    y_probs = gbm.predict(X_test_pca, num_iteration=gbm.best_iteration)

    auc = roc_auc_score(y_test, y_probs)
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

print('Best trial:', study.best_trial)
print('Best AUC:', study.best_value)
print('Best hyperparameters:', study.best_params)

[I 2025-05-10 17:31:07,108] A new study created in memory with name: no-name-9dc34cb1-1d31-45fa-957d-78b0bec9744f
[I 2025-05-10 17:31:09,052] Trial 0 finished with value: 0.5991305697873003 and parameters: {'n_components': 0.8984189063618768, 'num_leaves': 77, 'learning_rate': 0.0017625091375854279, 'max_depth': 3}. Best is trial 0 with value: 0.5991305697873003.
[I 2025-05-10 17:31:12,489] Trial 1 finished with value: 0.6113569321533924 and parameters: {'n_components': 0.8452766054959978, 'num_leaves': 6, 'learning_rate': 0.002547414540880566, 'max_depth': 5}. Best is trial 1 with value: 0.6113569321533924.
[I 2025-05-10 17:31:14,328] Trial 2 finished with value: 0.6095326812606738 and parameters: {'n_components': 0.805497452682624, 'num_leaves': 115, 'learning_rate': 0.006054725689408893, 'max_depth': 11}. Best is trial 1 with value: 0.6113569321533924.
[I 2025-05-10 17:31:16,316] Trial 3 finished with value: 0.6146561092997982 and parameters: {'n_components': 0.897020412714159, 'num

Best trial: FrozenTrial(number=110, state=1, values=[0.637090513895358], datetime_start=datetime.datetime(2025, 5, 10, 17, 34, 29, 42646), datetime_complete=datetime.datetime(2025, 5, 10, 17, 34, 30, 645036), params={'n_components': 0.8416698288987298, 'num_leaves': 130, 'learning_rate': 1.6784444229130393e-05, 'max_depth': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_components': FloatDistribution(high=0.9, log=False, low=0.8, step=None), 'num_leaves': IntDistribution(high=150, log=False, low=5, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-06, step=None), 'max_depth': IntDistribution(high=12, log=False, low=3, step=1)}, trial_id=110, value=None)
Best AUC: 0.637090513895358
Best hyperparameters: {'n_components': 0.8416698288987298, 'num_leaves': 130, 'learning_rate': 1.6784444229130393e-05, 'max_depth': 4}
